# 14.3.2 AlexNet

**AlexNet (2012):** $\;$ CNN de Alex Krizhevsky para ImageNet
* Redujo el error del $26\%$ al $15\%$ (en top-5 para 1000 clases)
* Comparación con LeNet:
    * Más profunda: 8 capas de parámetros ajustables en lugar de 5
    * Función de activación: ReLU en lugar de tanh
    * Regularización: dropout en lugar de weight decay
    * Apila varias capas convolucionales sin alternarlas con pooling
<div align="center">
<table><tr>
<td style="border: none;"><img src="LeNetAlexNet.png" width=800/></td>
</tr></table>
</div>